In [ ]:
import redis
import json

# Define connection variables
host = 'redis-13857.c14.us-east-1-2.ec2.redns.redis-cloud.com'
port = 13857
password = 'okNaqK7r5dKE6stSis0FlUuiDlhH2t7M'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

# Real-time Product Quality Monitoring

## Description of the use case
This use case involves monitoring real-time product quality metrics using Redis Count-Min Sketch. By tracking quality-related events, businesses can quickly identify and address quality issues.

## Redis Data Structures Used
- **Count-Min Sketch**: To monitor real-time product quality metrics.
- **Hashes**: To store additional information about products and quality metrics.


In [ ]:
# Mock data for the use case
quality_data = {
    'product_1001': {'name': 'Product A', 'defects': 0},
    'product_1002': {'name': 'Product B', 'defects': 0},
    'product_1003': {'name': 'Product C', 'defects': 0},
    'product_1004': {'name': 'Product D', 'defects': 0}
}

# Use Hashes to store product and quality information
for product_id, data in quality_data.items():
    r.hset(product_id, mapping=data)

# Initialize Count-Min Sketch for tracking defects
r.cms().initbydim(key='quality_metrics', width=2000, depth=5)

# Function to update quality metrics
def update_quality_metrics(product_id, defects):
    r.cms().incrby('quality_metrics', items=product_id, increments=defects)
    current_defects = r.hincrby(product_id, 'defects', defects)
    return current_defects

# Update quality metrics for products
update_quality_metrics('product_1001', 5)
update_quality_metrics('product_1002', 3)
update_quality_metrics('product_1003', 8)
update_quality_metrics('product_1004', 2)

# Function to get quality metrics
def get_quality_metrics(product_id):
    return r.cms().query('quality_metrics', product_id)

# Get and print quality metrics for each product
for product_id in quality_data.keys():
    defects = get_quality_metrics(product_id)
    print(f'Defects for {product_id}: {defects}')

# Print the product quality data
for product_id in quality_data.keys():
    print(f"{product_id}: {r.hgetall(product_id)}")
